In [1]:
prompts = [
    "An image of a pointling animation character swimming at the beach",
    "An image of a pointling animation character playing in snow",
    "An image of a pointling animation character playing guitar",
    "An image of a pointling animation character going to school",
    "An image of a pointling animation character on a roller coaster",
    "An image of a pointling animation character standing at a queue",
    "An image of a pointling animation character yelling help",
    "An image of a pointling animation character catcthing a plane",
    "An image of a pointling animation character rolling down the hill",
    "An image of a pointling animation character playing baseball"
]
from diffusers.utils import load_image


/home/jrajan/micromamba/envs/diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch


pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

model_path = "/home/jrajan/dreambooth/models/pointling_lora"
pipe.unet.load_attn_procs(model_path)




Loading pipeline components...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.35s/it]


In [26]:
images = pipe(prompts, num_inference_steps=10, guidance_scale=10, width=512, height=512,  output_type="np").images
print(images.shape)

from torchmetrics.functional.multimodal import clip_score
from functools import partial

clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(clip_score), 4)

sd_clip_score = calculate_clip_score(images, prompts)
print(f"CLIP score: {sd_clip_score}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.44it/s]


(10, 512, 512, 3)
CLIP score: 29.8286
